### 1) Load necessary Python packages

In [ ]:
import os
import sys
import dill

# Add the local src directory to the path
sys.path.append('./src/')

# Load the GBA_model class
from GBA_model import *

### 2) Define a function to load a model from a binary backup
See the script `csv_to_binary.py`.

In [ ]:
### Load the model from a binary file ###
def load_model( model_name ):
    filename = "./binary_models/"+model_name+".gba"
    assert os.path.isfile(filename), "ERROR: model not found."
    ifile = open(filename, "rb")
    model = dill.load(ifile)
    ifile.close()
    return model

### 3) Load the model "EC12b" (a very abstract model of <em>E. coli</em>)

In [ ]:
# I previously saved the model as a binary file with the script `csv_to_binary.py`
model = load_model("D")

### 4) Define the condition
The condition defines the environmental state of the cell (the composition of the medium). External concentrations are stored in the vector $x$.
Conditions are listed for the EC12b model in `./csv_models/EC12b/conditions.csv`.
We will use the condition 1.

In [ ]:
model.set_condition("1")

In [ ]:
model.solve_local_linear_problem()

### 6) Calculate all model variables

In [ ]:
model.calculate()

In [ ]:
def saveValues(model):
  dict_arrays = {
    "Max_growthrate": model.mu,
    "F-Vector": model.f,
    "Protein_concentrations vector" : model.p,
    "GCC_F": model.GCC_f,
    "Fluxes_vector" : model.v,
    "Internal_Metabolite_concentrations": model.c,
    "External_metabolite_concentrations": model.x,
    "Metabolite_concentrations": model.xc,
  }
  dict_arrays_str = {k: [str(v)] for k, v in dict_arrays.items()}

  # Erstellen des DataFrames
  df = pd.DataFrame(dict_arrays_str)

  # Speichern des DataFrames als CSV
  df.to_csv("Values_at_Max.csv", sep=',', index=False)

  print(df)

In [ ]:
import matplotlib.pyplot as plt
def plotTrajectory(timestamps, muRates):
  # Daten für die Zeitachse
  t = timestamps  # 100 Zeitpunkte von 0 bis 10

  # Daten für die Y-Achse (Beispiel: Sinusfunktion)
  mu = muRates

  # Erstellung des Plots
  plt.figure(figsize=(8, 6))  # Größe des Diagramms festlegen
  plt.plot(t, mu, label='mu(t)')  # Plot der Daten
  plt.xlabel('Time')  # Beschriftung der X-Achse
  plt.ylabel('mu')    # Beschriftung der Y-Achse
  plt.title('Plot of mu against Time')  # Titel des Diagramms
  plt.grid(True)      # Gitterlinien aktivieren
  plt.legend()        # Legende hinzufügen
  plt.show()          # Diagramm anzeigen
  print("max μ rate :")
  print(np.max(mu))
  return 

In [ ]:
def trajectory(model,max_time=1000,first_dt = 0.01,dt_changeRate=0.1):
                    # for gradient "=" 0
  dt = first_dt
  t = 0                              # time
  print(TRAJECTORY_STABLE_MU_COUNT)
  previous_mu = model.mu
  mu_alterationCounter = 0
  consistent_f = np.copy(model.f_trunc) # safes consistent_f
  next_f = np.copy(model.f_trunc)     # the f_trunc, that we are going to change
  allGCC_F = [model.GCC_f[1:]]       # to collect all previous GCC_f (just for checking the change of GCC_f)
  #print("first next_f: ")
  #print(next_f)
  #print("first GCC_f: ")
  #print(model.GCC_f)
  y_muRates = []                      #save muRates for plotting
  timestamps = []                     #save timeStamps for plotting
  while (t < max_time):                                                                 # end loop if time is up

    if( ( np.abs(model.GCC_f) <= TRAJECTORY_CONVERGENCE_TOL ).all() and model.consistent):               #(maybe use mu and look if it doesnt change for the next x steps , stop)
      #print("absolute of GCC_F is less then threshold in GCC_F :", model.GCC_f)
      break

    if(model.mu == previous_mu):
      mu_alterationCounter = mu_alterationCounter + 1
      #print(mu_alterationCounter)
    else:
        mu_alterationCounter = 0

    if(mu_alterationCounter >= TRAJECTORY_STABLE_MU_COUNT):
        break
    if np.any(next_f < 0):                                                            #negative value correction
       #print("next_f before neg.correction:", next_f)
       next_f[next_f < 0] = 1e-10
       #print("next_f after neg.correction:", next_f)

    print("current gradient :", model.GCC_f[-1])
    print("current protein",model.p)

    next_f = np.add(next_f, model.GCC_f[1:] * dt)                                      # add without first index of GCC_f

    #print(f"np.add({next_f}, {model.GCC_f[1:]} * {dt})")                                  # shows the term sum of next_f and GCC_f *dt

    model.set_f(next_f)
    model.calculate()                                                             #calculate everything
    model.check_model_consistency()                                               #check consistency
    #allGCC_F = np.vstack((allGCC_F,model.GCC_f[1:]))
    if (model.consistent):
      timestamps = np.append(timestamps,t)
      y_muRates = np.append(y_muRates,model.mu)
      #print(y_muRates)
      t = t + dt                                                                  # calc. new t
      consistent_f = next_f                                                       # saves new f as the consistent f

    else:
      next_f = consistent_f                                                       #resets next_f to last consistent_f
      model.set_f(consistent_f)

      if (dt > 1e-100):                                                           # make sure dt is not too small
       dt = dt * dt_changeRate
    
       t = t + dt                                                                 # calc. new t
      else:
        break
      
  plotTrajectory(timestamps, y_muRates)
  saveValues(model)
  #print(timestamps, y_muRates, model.consistent)
  #print('Last GCC_f')
  #print(model.GCC_f)
  #print(allGCC_F)
  
  return 

trajectory(model, max_time=200, first_dt = 0.01, dt_changeRate=0.1)

